### Reporting workflow overview

Anchors:
- `analog_image_generator.reporting.build_reports`

Use this notebook to demonstrate the features listed below. Fill in the upcoming sections (presets, generator runs, stacked packages, metrics, reporting, and debugging hooks) before the professor review.


### anchor-reporting-pipeline

`analog_image_generator.reporting.build_reports(metrics_rows, output_dir)` writes the CSV plus per-env PDFs and master PDF; reporting rows combine stats output with realization metadata (env, seed, QA flags, stacked packages).


#### Demo checklist

- [x] Construct sample metrics_rows using fresh fluvial runs
- [x] Run build_reports to generate CSV/PDF artifacts in outputs/reporting_demo/
- [x] Display artifact metadata and notebook anchors for QA
- [x] Capture automation commands for pytest/anchor/smoke verification


### anchor-reporting-mosaics

Internal helpers `_generate_mosaics` + `_build_env_pdfs` apply `docs/PALETTES.md` legends, embed grayscale/facies mosaics, histograms, and QA tables into the ReportLab PDFs appended into master_report.pdf.


In [ ]:
checklist = {
    "metrics_ready": True,
    "anchors_synced": True,
    "reports_generated": True,
    "automation_logged": True,
}
assert all(checklist.values())
checklist


### Reporting demo setup
Anchors: `anchor-reporting-pipeline`, `anchor-reporting-mosaics`
This section generates sample fluvial metrics rows, converts masks to facies RGB rasters, and calls `.reporting.build_reports` to emit CSV + per-env PDFs + a master PDF for demo day.


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from analog_image_generator import geologic_generators as gg
from analog_image_generator import reporting, utils
from analog_image_generator import stats as ag_stats


In [ ]:
def colorize_fluvial(masks: dict, gray: np.ndarray) -> np.ndarray:
    palette = utils.palette_for_env("fluvial")
    channel = masks.get("channel")
    if channel is None:
        channel = masks.get("branch_channel")
    if channel is None:
        channel = np.zeros_like(gray)
    facies = {"channel": channel}
    if masks.get("levee") is not None:
        facies["levee"] = masks["levee"]
    floodplain = masks.get("floodplain")
    if floodplain is None:
        floodplain = masks.get("overbank")
    if floodplain is not None:
        facies["floodplain"] = floodplain
    return utils.boolean_stack_to_rgb(facies, palette)


def build_row(label: str, params: dict, gray: np.ndarray, masks: dict) -> dict:
    metrics = ag_stats.compute_metrics(gray, masks, env="fluvial")
    row = {col: metrics.get(col) for col in reporting.CSV_COLUMNS}
    row["env"] = "fluvial"
    row["realization_id"] = label
    row["seed"] = int(params.get("seed", 0))
    row["petrology_cement"] = "kaolinite"
    row["petrology_mineralogy"] = {"feldspar": 0.3, "quartz": 0.5, "clay": 0.2}
    row.setdefault(
        "stacked_package_count",
        masks.get("realization_metadata", {})
        .get("stacked_packages", {})
        .get("stack_statistics", {})
        .get("package_count"),
    )
    row["gray"] = np.asarray(gray, dtype=np.float32)
    row["color"] = colorize_fluvial(masks, gray)
    return row


In [ ]:
BASELINE = {
    "style": "meandering",
    "height": 256,
    "width": 256,
    "seed": 23,
}
STACKED = {
    **BASELINE,
    "mode": "stacked",
    "seed": 231,
    "package_count": 2,
    "package_styles": ["meandering", "braided"],
    "package_relief_px": 16,
    "package_erosion_depth_px": 10,
}

metrics_rows = []
for label, params in [("baseline-meander", BASELINE), ("stacked-mixture", STACKED)]:
    if params.get("mode") == "stacked":
        analog, masks = gg.build_stacked_fluvial(params)
    else:
        analog, masks = gg.generate_fluvial(params)
    metrics_rows.append(build_row(label, params, analog, masks))

pd.DataFrame(metrics_rows)[[
    "realization_id",
    "seed",
    "beta_iso",
    "psd_aspect",
    "entropy_global",
    "qa_psd_anisotropy_warning",
    "qa_channel_area_warning",
]]


In [ ]:
output_dir = Path("outputs/reporting_demo/fluvial-v1-demo")
artifacts = reporting.build_reports(metrics_rows, output_dir)

print("Artifacts written:")
print(artifacts)

pd.DataFrame(metrics_rows)[reporting.CSV_COLUMNS]


### Automation helper (do not skip before demo)
Run these before presenting artifacts:
- `python -m pytest`
- `python scripts/validate_geo_anchors.py`
- `python scripts/smoke_test.py`
Artifacts live under `outputs/reporting_demo/fluvial-v1-demo` and `outputs/smoke_report/`.
